## Stock Analysis Engine

Analyze information about publicly traded companies from Yahoo (https://finance.yahoo.com/) and IEX Real-Time Price https://iextrading.com/developer/docs/ (supported data includes: news, quotes, dividends, daily, intraday, statistics, financials, earnings, options, and more). Once collected the data is archived in s3 (using minio https://minio.io) and automatically cached in redis.

It uses Celery workers to process all tasks http://www.celeryproject.org/ and is a horizontally scalable worker pool that works with many transports and backends (https://github.com/celery/celery#transports-and-backends)


## Getting Started

### Start Services

To develop jupyter notebooks start the notebook-integration (https://github.com/AlgoTraders/stock-analysis-engine/blob/master/compose/notebook-integration.yml) containers using docker-compose. Here's the command to start it:

```
./compose/start.sh -j
```

Verify the containers are running:

```
docker ps -a
```

### Download and Prepare Data

With a valid internet connection you can the following download datasets from IEX and Yahoo. 

#### IEX via pyEX

- minute (intraday)
- daily
- stats
- peers
- news
- financials
- earnings
- dividends
- company

#### Yahoo

- pricing
- options
- news

Note, please refer to the latest data that is fetched for these services:

- IEX - https://github.com/AlgoTraders/stock-analysis-engine/blob/master/analysis_engine/iex/fetch_data.py
- Yahoo - https://github.com/AlgoTraders/stock-analysis-engine/blob/master/analysis_engine/yahoo/get_data.py

### Load 

In [18]:
import datetime
import analysis_engine.work_tasks.get_new_pricing_data as task_pricing
from analysis_engine.api_requests import build_get_new_pricing_request
from spylunking.log.setup_logging import build_colorized_logger

log = build_colorized_logger(name='intro-nb')

### Set up Collection

In [23]:
ticker = 'SPY'

# Store in this S3 key:
s3_bucket = 'tutorials'
s3_key = '{}_intro_{}'.format(ticker, datetime.datetime.now().strftime('%Y-%m-%d'))

# Store in this Redis key
redis_key = '{}_latest'.format(ticker)

"""
Build the request (a work dictionary):
https://github.com/AlgoTraders/stock-analysis-engine/blob/master/analysis_engine/api_requests.py#L49
"""
work = build_get_new_pricing_request()

work['ticker'] = ticker
work['fetch_mode'] = 'all'
work['s3_key'] = s3_key
work['redis_key'] = redis_key
work['label'] = '{}'.format(s3_key)
work['celery_disabled'] = True

log.info(
    'starting collection ticker={} '
    'redis_key={} s3_bucket={} s3_key={}'.format(
        ticker,
        redis_key,
        s3_bucket,
        s3_key))

2018-10-05 06:02:57,206 - intro-nb - INFO - starting collection ticker=SPY redis_key=SPY_latest s3_bucket=tutorials s3_key=SPY_intro_2018-10-05


In [24]:
task_res = task_pricing.get_new_pricing_data(
    work)

2018-10-05 06:02:59,237 - analysis_engine.work_tasks.get_new_pricing_data - INFO - task - get_new_pricing_data - start work_dict={'ticker': 'SPY', 'ticker_id': 1, 's3_bucket': 'pricing', 's3_key': 'SPY_intro_2018-10-05', 'redis_key': 'SPY_latest', 'strike': None, 'contract': 'C', 'get_pricing': True, 'get_news': True, 'get_options': True, 'fetch_mode': 'all', 'label': 'SPY_intro_2018-10-05', 'celery_disabled': True}
2018-10-05 06:02:59,244 - analysis_engine.work_tasks.get_new_pricing_data - INFO - SPY_intro_2018-10-05 getting pricing for ticker=SPY cur_date=2018-10-05 06:02:59.244627 exp_date=2018-10-19 06:02:59.244637
2018-10-05 06:02:59,250 - analysis_engine.work_tasks.get_new_pricing_data - INFO - SPY_intro_2018-10-05 getting ticker=SPY pricing
2018-10-05 06:03:00,061 - analysis_engine.work_tasks.get_new_pricing_data - INFO - SPY_intro_2018-10-05 ticker=SPY close=291.72 vol=111545910
2018-10-05 06:03:00,064 - analysis_engine.work_tasks.get_new_pricing_data - INFO - SPY_intro_2018-10

In [26]:
log.info('Intro ticker={} completed'.format(
    ticker))

2018-10-05 06:04:48,782 - intro-nb - INFO - Intro ticker=SPY completed
